In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_selection import RFE

import statsmodels.api as sm  
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.tsa.api as smt
import scipy as sp

import warnings
warnings.filterwarnings('ignore')

**Reading the data**

In [2]:
df  = pd.read_csv('day.csv')
print(df.shape)
df.head()

(730, 16)


,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,01-01-2018,1,0,1,0,6,0,2,14.110847,18.18125,80.5833,10.749882,331,654,985
1,2,02-01-2018,1,0,1,0,0,0,2,14.902598,17.68695,69.6087,16.652113,131,670,801
2,3,03-01-2018,1,0,1,0,1,1,1,8.050924,9.47025,43.7273,16.636703,120,1229,1349
3,4,04-01-2018,1,0,1,0,2,1,1,8.200000,10.60610,59.0435,10.739832,108,1454,1562
4,5,05-01-2018,1,0,1,0,3,1,1,9.305237,11.46350,43.6957,12.522300,82,1518,1600


## **Data Visualisation** 
Lets Summarize the data

In [3]:
df.describe()

,instant,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
count,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000
mean,365.500000,2.498630,0.500000,6.526027,0.028767,2.997260,0.683562,1.394521,20.319259,23.726322,62.765175,12.763620,849.249315,3658.757534,4508.006849
std,210.877136,1.110184,0.500343,3.450215,0.167266,2.006161,0.465405,0.544807,7.506729,8.150308,14.237589,5.195841,686.479875,1559.758728,1936.011647
min,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,2.424346,3.953480,0.000000,1.500244,2.000000,20.000000,22.000000
25%,183.250000,2.000000,0.000000,4.000000,0.000000,1.000000,0.000000,1.000000,13.811885,16.889713,52.000000,9.041650,316.250000,2502.250000,3169.750000
50%,365.500000,3.000000,0.500000,7.000000,0.000000,3.000000,1.000000,1.000000,20.465826,24.368225,62.625000,12.125325,717.000000,3664.500000,4548.500000
75%,547.750000,3.000000,1.000000,10.000000,0.000000,5.000000,1.000000,2.000000,26.880615,30.445775,72.989575,15.625589,1096.500000,4783.250000,5966.000000
max,730.000000,4.000000,1.000000,12.000000,1.000000,6.000000,1.000000,3.000000,35.328347,42.044800,97.250000,34.000021,3410.000000,6946.000000,8714.000000


Data Types of columns

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 730 entries, 0 to 729
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   instant     730 non-null    int64  
 1   dteday      730 non-null    object 
 2   season      730 non-null    int64  
 3   yr          730 non-null    int64  
 4   mnth        730 non-null    int64  
 5   holiday     730 non-null    int64  
 6   weekday     730 non-null    int64  
 7   workingday  730 non-null    int64  
 8   weathersit  730 non-null    int64  
 9   temp        730 non-null    float64
 10  atemp       730 non-null    float64
 11  hum         730 non-null    float64
 12  windspeed   730 non-null    float64
 13  casual      730 non-null    int64  
 14  registered  730 non-null    int64  
 15  cnt         730 non-null    int64  
dtypes: float64(4), int64(11), object(1)
memory usage: 91.4+ KB


Check for Null Values

In [5]:
df.isna().sum()


instant       0
dteday        0
season        0
yr            0
mnth          0
holiday       0
weekday       0
workingday    0
weathersit    0
temp          0
atemp         0
hum           0
windspeed     0
casual        0
registered    0
cnt           0
dtype: int64

No Null values present in the data.<br>

We can remove instant,dteday,casual and Registered as cnt is sum of casual and registered and instant is index and we got all details from dteday.

In [6]:
df.drop(columns = ["instant","dteday","casual","registered"],inplace =True)

Let's look at the pair plot of thw whole data

In [ ]:
sns.pairplot(df)

As we can observe the pairplot is too clumsy lets, separate the numerical and categorical variables

In [ ]:
df.columns

In [ ]:
cat = ["season","yr","mnth","holiday","weekday","workingday","weathersit"]
num = ["temp","atemp","hum","windspeed"]
df['weathersit'] = df['weathersit'].map({1:'Clear', 2:'Cloudy', 3:'LSLR', 4:'HRHS'})

Lets look at the pairplot of only Numerical variables.

In [ ]:
sns.pairplot(df[num+["cnt"]])

We can observe that temp and atemp are higly correlated to each other, and these two variables are also related to count.

Now let us observe the effect of categorical variables.

In [ ]:
plt.figure(figsize=(20, 12))
plt.subplot(3,3,1)
sns.boxplot(x = cat[0], y = 'cnt', data = df)
plt.subplot(3,3,2)
sns.boxplot(x = cat[1], y = 'cnt', data = df)
plt.subplot(3,3,3)
sns.boxplot(x = cat[2], y = 'cnt', data = df)
plt.subplot(3,3,4)
sns.boxplot(x = cat[3], y = 'cnt', data = df)
plt.subplot(3,3,5)
sns.boxplot(x = cat[4], y = 'cnt', data = df)
plt.subplot(3,3,6)
sns.boxplot(x = cat[5], y = 'cnt', data = df)
plt.subplot(3,3,7)
sns.boxplot(x = cat[6], y = 'cnt', data = df)
plt.show()


* From the graphs we can observe that weekday and workingday has no effect on CNT variable.
* We can observe that the median of the yr in 2019 is greater than 2018 which says the cnt variable is more in year 2019.
* We can also observe that there is some pattern in month, Season and weathersit vaariable's.
* We can see on holidays bike count is less.

Lets See the Correlation Heat-map

In [ ]:
plt.figure(figsize = (30, 15))
sns.heatmap(df.corr(), annot = True)
plt.show()

* From the Heat map we can clearly see that season and month are higly correlated.
* Cnt is correlated with yr,temp and atemp.
* Temp and atemp are higly correlated with 0.99 so we can clearly have to remove one variable.


## **Data Preparation**

Lets Decrypt the categorical variables Season and weather as per readme.txt and create dummy variables for these two variables.

In [ ]:

def season_enc(x):
    return x.map({1: "Spring", 2: "summer",3:"fall",4:"winter"})
def weather_enc(x):
    return x.map({1:"Clear",2:"Mist",3:"Light Snow",4:"Heavy Rain"})

enc_df = df.copy()
enc_df[["season"]] = enc_df[["season"]].apply(season_enc)
enc_df[["weathersit"]] = enc_df[["weathersit"]].apply(weather_enc)
enc_df.head()

In [ ]:
temp = pd.get_dummies(enc_df[["season","weathersit"]],drop_first="True")
temp.head()

Lets join the dummy variables to our enc_df and drop the original variables.

In [ ]:
enc_df = pd.concat([enc_df, temp], axis = 1)
enc_df.head()

In [ ]:
enc_df.drop(columns = ["season","weathersit"], axis = 1, inplace = True)
enc_df.head()

We can see that temp and atemp are higly correlated so we need to remove one variable, Lets calculate the VIF an decide which variable to remove.

Let's Divide the data into training and test set.

In [ ]:
df_train, df_test = train_test_split(enc_df, train_size = 0.7, test_size = 0.3, random_state = 100)

Lets Scale the Numerical variables.

In [ ]:
num

In [ ]:
scaler = MinMaxScaler()
df_train[num+["cnt"]] = scaler.fit_transform(df_train[num+["cnt"]])

In [ ]:
df_train.describe()

Lets Divide the data into independent and dependent Variables.

In [ ]:
y_train = df_train.pop('cnt')
X_train = df_train

## ***Data Modeling and Evaluation***

Lets store X_train data frame in X_train_copy as backup.

In [ ]:
X_train_copy = X_train.copy()

Lets see the correlation of the newly formed variables.

In [ ]:
plt.figure(figsize = (30, 15))
sns.heatmap(df_train.corr(), annot = True)
plt.show()

In [ ]:
vif = pd.DataFrame()
vif['Features'] = X_train.columns
vif['VIF'] = [variance_inflation_factor(X_train.values, i) for i in range(X_train.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

Now we will drop the temp variable as its VIF is greater than all.

In [ ]:
X_train.drop(columns=["temp"],inplace=True)


Lets store all the confirmed delete variables in conf_del_cols list to keep track of deleted variables.

In [ ]:
conf_del_cols = ["temp"]

Now we will Observe the VIF values after deleting temp.

In [ ]:
vif = pd.DataFrame()
vif['Features'] = X_train.columns
vif['VIF'] = [variance_inflation_factor(X_train.values, i) for i in range(X_train.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

Let's Create a model with all the remaining variables.

In [ ]:
X_train_lm = sm.add_constant(X_train)
lr_1 = sm.OLS(y_train, X_train_lm).fit()
print(lr_1.summary())

We Observe that mnth variable is having high P-value and high VIF , some can drop this variable.

In [ ]:
X_train.drop(columns=["mnth"],inplace=True)

In [ ]:
X_train_lm = sm.add_constant(X_train)

lr_2 = sm.OLS(y_train, X_train_lm).fit()

print(lr_2.summary())


We Observe that The R-square value Adjusted R-square value Doesnt change so we can say that mnth doesnot have any effect on model.

In [ ]:
conf_del_cols.append("mnth")

In [ ]:
vif = pd.DataFrame()
vif['Features'] = X_train.columns
vif['VIF'] = [variance_inflation_factor(X_train.values, i) for i in range(X_train.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

Lets Drop atemp variable because it's having high VIF value, And see how model behaves.

In [ ]:
X_train.drop(columns=["atemp"],inplace=True)

In [ ]:
X_train_lm = sm.add_constant(X_train)

lr_3 = sm.OLS(y_train, X_train_lm).fit()

print(lr_3.summary())

We can see that by dropping atemp we saw the decrease in R^2 and Adj R^2 value so we should not drop that column.
<br><br> Lets backup the data.

In [ ]:
X_train = X_train_copy.copy()
X_train.drop(columns=conf_del_cols,inplace=True)

Lets use **RFE** to select top features to build the model.


In [ ]:
lm_rfe = LinearRegression()
lm_rfe.fit(X_train, y_train)

rfe = RFE(lm_rfe)         
rfe = rfe.fit(X_train, y_train)

In [ ]:
list(zip(X_train.columns,rfe.support_,rfe.ranking_))

Lets take only the column values with ranking as 1

In [ ]:
X_train = X_train_copy[["yr","atemp","hum","windspeed","season_winter","weathersit_Light Snow"]]

In [ ]:
X_train_lm = sm.add_constant(X_train)

lr_5 = sm.OLS(y_train, X_train_lm).fit()

print(lr_5.summary())

We can Clearly see that all the p values are small enough to reject the null hypothesis<br><br>
Now lets see the VIF values of these selected variables.

In [ ]:
vif = pd.DataFrame()
vif['Features'] = X_train.columns
vif['VIF'] = [variance_inflation_factor(X_train.values, i) for i in range(X_train.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

Lets drop the Hum variable as it is having highest VIF nearer to 10.

In [ ]:
X_train.drop(columns=["hum"],inplace=True)

Lets build the model again.

In [ ]:
X_train_lm = sm.add_constant(X_train)

lr_6 = sm.OLS(y_train, X_train_lm).fit()

print(lr_6.summary())

Lets Check VIF Values

In [ ]:
vif = pd.DataFrame()
vif['Features'] = X_train.columns
vif['VIF'] = [variance_inflation_factor(X_train.values, i) for i in range(X_train.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

We can observe that all the variables are having VIF < 5 which is good enough to go a head.

###  Residual Analysis

In [ ]:
y_pred = lr_6.predict(X_train_lm)

Lets Draw the histogram of errors

In [ ]:
fig = plt.figure()
residual = y_train - y_pred
sns.distplot(residual, bins = 20)
fig.suptitle('Error Terms', fontsize = 20)            
plt.xlabel('Errors', fontsize = 18)  
plt.show()

As we can see that error terms are following Normal Distribution with mean as 0(Zero).

Lets Plot the q-q Plot 

In [ ]:
fig, ax = plt.subplots(figsize=(6,2.5))
_, (__, ___, r) = sp.stats.probplot(residual, plot=ax, fit=True)

We can Clearly see that the points are forming straight line which says booth set of quantiles came rom the same distribution.

Lets Draw the Scatter Plot of Residuals

In [ ]:
s = sns.scatterplot(y_pred,residual)
s.set_title("y_pred vs residual")
s.set(xlabel='y_pred', ylabel='residual')
s.axhline(0)
plt.show()

We can clearly see that the graph is representing **Homoscedasticity**

Lets Draw the Auto correlation factor graph for the Residuals.

In [ ]:
acf = smt.graphics.plot_acf(residual, lags=40 , alpha=0.05)

We can see that the data is having randomness.

### Model Evaluation

Lets use our Scalar fitted with train data to transform test data.

In [ ]:
df_test[num+["cnt"]] = scaler.transform(df_test[num+["cnt"]])

In [ ]:
df_test.describe()

Lets Divide scaled data intoo X_test and y_test.

In [ ]:
y_test = df_test.pop('cnt')
X_test = df_test

Now only select the columns that we used in our final modeland store the data in X_test_new. 

In [ ]:
X_test_new = X_test[X_train.columns]

Add the constant.

In [ ]:
X_test_new = sm.add_constant(X_test_new)

Lets get the predictions for the test data .

In [ ]:
y_pred = lr_6.predict(X_test_new)

Now we will draw scatter plot between y_test and y_pred to see weather they are following the pattern or not.

In [ ]:
s = sns.scatterplot(y_test,y_pred)
s.set_title("y_test vs y_pred")
s.set(xlabel='y_test', ylabel='y_pred')
plt.show()

From the graph we can clearly see that y_pred and y_test are following the pattern and they are mostly similar to each other.

Lets get the predictions for trainigdata also to see how well our model performed on **Training data vs Test Data** .

In [ ]:
y_train_pred = lr_6.predict(X_train_lm)

In [ ]:
print("Model Performance on Train Data Set")
print("Mean squared error: %.2f" % mean_squared_error(y_train, y_train_pred))
print("Coefficient of determination(R^2): %.2f" % r2_score(y_train, y_train_pred))
x = 1-(1-r2_score(y_train, y_train_pred))*(len(X_train)-1)/(len(X_train)-5-1)
print("Adjusted R^2: %.2f" % x)

In [ ]:
print("Model Performance on Test Data Set")
print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred))
print("Coefficient of determination(R^2): %.2f" % r2_score(y_test, y_pred))
x = 1-(1-r2_score(y_test, y_pred))*(len(X_test)-1)/(len(X_test)-5-1)
print("Adjusted R^2: %.2f" % x)


We can see that the model performance on test data set is almost similar to performance on train data set.

Lets Plot the Actual and Predicted values of cnt for both Test and Train data.

In [ ]:
c = [i for i in range(1,len(X_test)+1,1)]
fig = plt.figure(figsize=(20,7))
plt.plot(c,y_test, color="yellow",  linewidth=2.5, linestyle="-",label="Actual")
plt.plot(c,y_pred, color="green", linewidth=2.5, linestyle="-",label="Predicted")

fig.suptitle('Actual and Predicted', fontsize=20)             
plt.xlabel('Index', fontsize=18)                               
plt.ylabel('cnt', fontsize=16)   
plt.legend()
plt.show()

In [ ]:
c = [i for i in range(1,len(X_train)+1,1)]
fig = plt.figure(figsize=(20,7))

plt.plot(c,y_train, color="yellow",  linewidth=2.5, linestyle="-",label="Actual")
plt.plot(c,y_train_pred, color="green", linewidth=2.5, linestyle="-",label="Predicted")
fig.suptitle('Actual and Predicted', fontsize=20)             
plt.xlabel('Index', fontsize=18)                               
plt.ylabel('cnt', fontsize=16)    
plt.legend()
plt.show()

Lets Summarize the Final Model

In [ ]:
print(lr_6.summary())


We can see that the equation of our best fitted line is:

$ cnt =0.0892 + 0.2341  \times  yr + 0.6381  \times  atemp + 0.095 \times season winter - 0.12 \times windspeed - 0.02409 \times weathersit Light Snow $


Overall we have a good decent model, but we also acknowledge that we could do better. 

We have a couple of options:
1. Add new features (week of month,temp by month , etc.)
2. Build a non-linear model.